In [31]:
import json
import torch
from tqdm import tqdm
from torch import nn
from copy import deepcopy
from torch.utils.data import DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [24]:
device = "cuda"

In [25]:
checkpoint = "models/checkpoint-7-0.98840/"
tokenizer = GPT2Tokenizer.from_pretrained(checkpoint)
model = GPT2LMHeadModel.from_pretrained(checkpoint)
tokenizer.pad_token = tokenizer.eos_token

In [37]:
valid = []
with open("data/process.train.json") as fin:
    for line in fin.readlines():
        valid.append(json.loads(line))

In [38]:
def tokenize(data):
    res = tokenizer(data, max_length=256, truncation=True, padding='max_length', return_tensors='pt')
    res['labels'] = deepcopy(res['input_ids'])
    return res

In [39]:
data = [tokenize(x['text']) for x in valid]
valid_dataloader = DataLoader(data, batch_size=1)

In [40]:
model.eval();
model.to(device);

In [41]:
examples = []

In [42]:
for step, batch in enumerate(tqdm(valid_dataloader)):
    with torch.no_grad():
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
    loss = outputs.loss
    examples.append((step, loss.item()))

100%|███████████████████████████████████████| 5000/5000 [01:18<00:00, 63.42it/s]


In [43]:
sorted(examples, key=lambda x: -x[1])

[(3124, 0.8081673383712769),
 (445, 0.7752009630203247),
 (2099, 0.7736316919326782),
 (4083, 0.7561647295951843),
 (1087, 0.7530858516693115),
 (2615, 0.744611918926239),
 (1797, 0.7346208095550537),
 (4469, 0.7298499941825867),
 (2379, 0.7235565781593323),
 (1045, 0.7178168892860413),
 (4292, 0.7171058654785156),
 (4508, 0.7138346433639526),
 (814, 0.7101637125015259),
 (226, 0.7001565098762512),
 (4079, 0.6950061917304993),
 (2045, 0.6904683113098145),
 (4495, 0.6854511499404907),
 (763, 0.6680291891098022),
 (1511, 0.6617342233657837),
 (3132, 0.6605417728424072),
 (1703, 0.6598057746887207),
 (4864, 0.6581703424453735),
 (4097, 0.6525073647499084),
 (2669, 0.6521302461624146),
 (3721, 0.6517570614814758),
 (2661, 0.6514726877212524),
 (743, 0.6513670086860657),
 (174, 0.6511381268501282),
 (3666, 0.6498242616653442),
 (1662, 0.648632824420929),
 (2642, 0.6453222632408142),
 (3483, 0.6406236886978149),
 (369, 0.6403462886810303),
 (4619, 0.6352337002754211),
 (3067, 0.6340904831886

In [44]:
valid[3124]

{'id': '609000',
 'text': '<sos_u> salve man, quero consultar dio/appice/grijalva/pilson minha placa que é qfg-5603 <eos_u><sos_b> [consulta_saldo][req_saldo][cumprimento][info_placa] placa qfg-5603 <eos_b><sos_a> [req_cpf] <eos_a><sos_r> Você pode me informar o seu CPF? <eos_r><sos_u> cpf é 105.370.296-86 <eos_u><sos_b> [consulta_saldo][info_cpf] cpf 105.370.296-86 placa qfg-5603 <eos_b><sos_a> [info_valor][req_mais] <eos_a><sos_r> Perfeito. O seu saldo é de [valor]. Ajudo em mais alguma coisa? <eos_r><sos_u> isso. preciso, obrigado. era só não <eos_u><sos_b> [negacao] cpf 105.370.296-86 placa qfg-5603 <eos_b><sos_a> [despedida] <eos_a><sos_r> Obrigado pelo contato. tenha uma boa tarde. <eos_r>'}

In [10]:
string = "<sos_u> boa noite, pode me dizer quanto de saldo eu tenho? <eos_u><sos_b> [consulta_saldo][req_saldo][cumprimento] <eos_b><sos_a> [req_cpf] <eos_a><sos_r> Sim, Claro. Qual é o seu cpf? <eos_r><sos_u> 978403690-03 <eos_u><sos_b> [consulta_saldo][info_cpf] cpf 978403690-03 <eos_b><sos_a> [req_placa] <eos_a><sos_r> Me informe sua placa, por favor? <eos_r><sos_u> nxi 4451 <eos_u><sos_b> [consulta_saldo][info_placa] cpf 978403690-03 placa nxi 4451 <eos_b><sos_a> [info_valor][req_mais] <eos_a><sos_r> o saldo é de [valor], mais alguma coisa? <eos_r><sos_u> e para a placa jgm1234<eos_u><sos_b>"
input_ids = tokenizer.encode(string)
eos_token = tokenizer.encode(['<eos_r>'])[0]
out = model.generate(input_ids=torch.tensor(input_ids).reshape(1,-1),
                                 pad_token_id=tokenizer.eos_token_id,
                                 max_length=len(input_ids)+60,
                                 eos_token_id=eos_token)

In [11]:
tokenizer.decode(out[0])

'<sos_u> <sos_u> boa noite, pode me dizer quanto de saldo eu tenho? <eos_u> <sos_b> [consulta_saldo] [req_saldo] [cumprimento] <eos_b> <sos_a> [req_cpf] <eos_a> <sos_r> Sim, Claro. Qual é o seu cpf? <eos_r> <sos_u> 978403690-03 <eos_u> <sos_b> [consulta_saldo] [info_cpf] cpf 978403690-03 <eos_b> <sos_a> [req_placa] <eos_a> <sos_r> Me informe sua placa, por favor? <eos_r> <sos_u> nxi 4451 <eos_u> <sos_b> [consulta_saldo] [info_placa] cpf 978403690-03 placa nxi 4451 <eos_b> <sos_a> [info_valor] [req_mais] <eos_a> <sos_r> o saldo é de [valor], mais alguma coisa? <eos_r> <sos_u> e para a placa jgm1234 <eos_u> <sos_b> [consulta_saldo] [info_placa] cpf 978403690-03 placa jgm1234 <eos_b> <sos_a> [info_valor] [req_mais] <eos_a> <sos_r> O saldo é de [valor]. Auxilio em mais alguma coisa? <eos_r>'